Stanislas Deneuville - Emmanuel Ferrandi - Pol Grisart - Marine Médard
# Project of data science :  Face recognition in a video and counting
16/11/2018

## Part II : Train&Use

In [1]:
import random
import keras
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

__split__ : split the set into two well mixed set 


In : 
* data : list of all the images 

Out : 
* train_set : list of the images in the training set representing 66% of data
* test_set : list of the images in the test set representing the other 34% of data

In [ ]:
TRAIN_TEST_RATIO = 0.66
def split(data:np.ndarray, train_test_ratio:int=TRAIN_TEST_RATIO, random_split=True) :
    if random_split:
        # Shuffle
        np.random.shuffle(data)
    
    # Split data
    cut_index = round(data.shape[0] * train_test_ratio) 
    train_set = data[:cut_index, :]
    test_set = data[cut_index:, :]
    return(train_set, test_set)

In [51]:
MAX_CATEGORY = 14

def equalize(subsets):
    # TODO
    return subsets

def load_and_split():
    subsets = []
    for nb_face in range(MAX_CATEGORY+1):
        # Y value of these alements
        categorical_y = np.zeros((1, MAX_CATEGORY+1))
        categorical_y[0, nb_face] = 1
        
        subsets.append(np.zeros((0, 50*50 + MAX_CATEGORY+1)))
        folder_path = os.path.join("train_set", str(nb_face))
        for filename in os.listdir(folder_path):
            # Filter non image files
            if ".jpeg" in filename or ".png" in filename:
                x = plt.imread(os.path.join(folder_path, filename)).reshape((1, 50*50))
                y = categorical_y
                xy = np.concatenate((x,y), axis=1)
                subsets[nb_face] = np.concatenate((subsets[nb_face], xy), axis=0)
    
    # Equalize to have the same number of each Y value
    equalized_subsets = equalize(subsets)
    
    # Split data respecting equalization
    train_set, test_set = split(equalized_subsets[0])
    for subset in equalized_subsets[1:]:
        add_train_set, add_test_set = split(subset)
        train_set = np.concatenate((train_set, add_train_set), axis=0)
        test_set = np.concatenate((test_set, add_test_set), axis=0)
    
    # Shuffle
    random.shuffle(train_set)
    random.shuffle(test_set)
    
    print("Train size = {}, test size = {}".format(train_set.shape, test_set.shape))
    
    return train_set, test_set
        

__train_neuural_network__ : function that creates a neural network and trains it with the train_set 

In [56]:
def generate_model():
    # Simple model
    model = keras.models.Sequential()
    
    # Add neurone layers
    model.add(keras.layers.Dense(units=500, activation='relu', input_dim=50*50))
    model.add(keras.layers.Dense(units=100, activation='relu'))
    model.add(keras.layers.Dense(units=40, activation='relu'))
    model.add(keras.layers.Dense(units=15, activation='relu'))
    
    # Learning process
    model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
    return model 

def train_neural_network(train_set):
    print("Generating model")
    model = generate_model()
    
    x_train = train_set[:, :50*50]
    y_train = train_set[:, 50*50:]
    
    print("Start training")
    model.fit(x_train, y_train, epochs=5, batch_size=32)
    
    return model

__evaluate_performance__ : function that applys the neural network on the images in the test_set and compare with the real number of faces in these images 

In [53]:
def evaluate_performance(model, test_set):
    
    x_test = test_set[:, :50*50]
    y_test = test_set[:, 50*50:]
    
    loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)
    

In [55]:
print("Split")
train_set, test_set = load_and_split()

print("Train")
model = train_neural_network(train_set)

print("Evaluate")
evaluate_performance(model, test_set)

Split
Train size = (157, 2515), test size = (76, 2515)
Train
Generating model
x train [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Start training
Epoch 1/5
157/157 [==============================] - 0s 3ms/step - loss: 6.9554 - acc: 0.1911
Epoch 2/5
157/157 [==============================] - 0s 2ms/step - loss: 5.1468 - acc: 0.4459
Epoch 3/5
157/157 [==============================] - 0s 2ms/step - loss: 2.8485 - acc: 0.6688
Epoch 4/5
157/157 [==============================] - 0s 2ms/step - loss: 2.2530 - acc: 0.8025
Epoch 5/5
157/157 [==============================] - 0s 2ms/step - loss: 1.9981 - acc: 0.8344
Evaluate
76/76 [==============================] - 0s 305us/step
